# Perceptron

In [1]:
import numpy as np

from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import iarray as ia
from iarray.udf import jit, Array, float64, int64

import matplotlib.pyplot as plt
%matplotlib inline

Create a synthetic dataset with 1000 observations and 10 features.

In addition, this dataset will form two clusters so that we can identify them with a neural network.

In [2]:
coord, cl = make_classification(1000, 10, random_state=0)
X, Xt, y, yt = train_test_split(coord, cl, test_size=0.3, random_state=0)

Define de neural network class

In [3]:
class NN(object):
    
    def __init__(self, input_layer, hidden_layer, output_layer, random_seed=0):
        ia.set_config(seed=0)
        self.l1_weights = ia.irandom.standard_normal(ia.DTShape((input_layer, hidden_layer)))
        self.l2_weights = ia.irandom.standard_normal(ia.DTShape((hidden_layer, output_layer)))

    def _sigmoid(self, x):
        return ia.expr_from_string("1 / (1 + exp(-x))", {"x": x}).eval()

    def _sigmoid_prime(self, x):
        return ia.expr_from_string("x * (1 - x)", {"x": x}).eval()
    
    def feed_forward(self, X):
        l1_output = ia.matmul(X, self.l1_weights)
        l1_output = self._sigmoid(l1_output)

        l2_output = ia.matmul(l1_output, self.l2_weights)
        l2_output = self._sigmoid(l2_output)

        return l1_output, l2_output
    
    def backpropagation(self, l1, l2, y):
        
        l2_error = (y - l2).eval()
        
        l2_delta = (l2_error * self._sigmoid_prime(l2)).eval()

        l1_error = ia.matmul(l2_delta, self.l2_weights.T)

        l1_delta = (l1_error * self._sigmoid_prime(l1)).eval()
        
        return l2_error, l1_delta, l2_delta

    def update_weights(self, X, l1, l1_delta, l2_delta, alpha=1.0):
        expr = f"w + {alpha} * d"
        d2 = ia.matmul(l1.T, l2_delta)
        d1 = ia.matmul(X.T, l1_delta)

        self.l2_weights = ia.expr_from_string(expr, {"w": self.l2_weights, "d": d2}).eval()
        self.l1_weights = ia.expr_from_string(expr, {"w": self.l1_weights, "d": d1}).eval()

    def predict(self, X):
        _, l2 = self.feed_forward(X)
        return l2

    def train(self, X, y, threshold=400):
        for j in range(threshold + 1):
            l1, l2 = self.feed_forward(X)
            l2_error, l1_delta, l2_delta = self.backpropagation(l1, l2, y)
            self.update_weights(X, l1, l1_delta, l2_delta, alpha=0.01)
            if(j % 100 == 0):
                train_error = ia.mean(ia.abs(l2_error).eval())
                print("epoch {:5} ".format(j),end='-')
                print(' error: {:0.4f} '.format(train_error))


Convert numpy datasets to iarray containers.

In [4]:
training_inputs = ia.numpy2iarray(X.astype(np.float64))
training_labels = ia.numpy2iarray(y.astype(np.float64).reshape(-1, 1))
inputs = ia.numpy2iarray(Xt.astype(np.float64))

Create a neural network with one hidden layer.

In [5]:
nn = NN(X.shape[1], 5, 1)

Train the neural network using the training dataset.

In [6]:
nn.train(training_inputs, training_labels, threshold=500)

epoch     0 - error: 0.3903 
epoch   100 - error: 0.1002 
epoch   200 - error: 0.0868 
epoch   300 - error: 0.0800 
epoch   400 - error: 0.0754 
epoch   500 - error: 0.0719 


Predict the test dataset using the fitted neural network.

In [7]:
labels = nn.predict(inputs)
labels = np.ravel((labels.data > 0.5).astype(int))

In [8]:
accuracy_score(labels, yt)

0.9533333333333334